In [119]:
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from typing import List, Tuple

In [120]:
# projection function definition

def projection_l1_builder(l: float = 1.0):
    def projection_l1(w: np.ndarray) -> np.ndarray:
        proj_w = w

        if np.linalg.norm(w, ord=1) > l:
            # solve the piecewise-linear equation by sorting
            sorted_abs_w = np.sort(np.abs(w))[::-1]
            for k in range(len(sorted_abs_w)):
                if (np.sum(sorted_abs_w[:k+1]) - l) / (k+1) >= sorted_abs_w[k]:
                    break
            else:
                k += 1

            tau = np.mean(sorted_abs_w[:k]) - l / k

            assert abs(np.sum(max(abs(wi) - tau, 0) for wi in w) / l - 1) < 1e-6, \
                f"w: {w}, " \
                f"sum: {np.sum(max(abs(wi) - tau, 0) for wi in w)}, " \
                f"tau: {tau}"
            
            # soft thresholding
            for i in range(len(proj_w)):
                if proj_w[i] > tau:
                    proj_w[i] -= tau
                elif proj_w[i] < -tau:
                    proj_w[i] += tau
                else:
                    proj_w[i] = 0

            assert np.linalg.norm(proj_w, ord=1) / l - 1 < 1e-6, np.linalg.norm(proj_w, ord=1)

        return proj_w

    return projection_l1


In [121]:
# objective and jacobian definition

def regression_builder():
    """
        data: np.ndarray, (n, d)
        labels: np.ndarray, (n,)
    """
    def regression_f(w, data, labels):
        res = np.dot(data, w) - labels              # (n,)
        f = 0.5 * np.linalg.norm(res)
        
        n_sample_feval = len(data)
        f_flop_counts = None

        return f, n_sample_feval, f_flop_counts

    def regression_J(w, data, labels):
        res = np.dot(data, w) - labels              # (n,)
        J = np.dot(res, data)
        
        n_sample_gradients = len(data)
        J_flop_counts = None

        return J, n_sample_gradients, J_flop_counts

    return regression_f, regression_J


In [122]:
# optimizer definition: pgd

def pgd(data: np.ndarray, labels: np.ndarray, w0,
        f, J, proj,
        lr, w_tol, f_tol, max_epochs) -> Tuple[List[np.ndarray], int]:
    assert data.shape[0] == labels.shape[0]
    
    # initialization
    ws = [proj(w0)]
    f0, n_sample_feval, f_flop_counts = f(ws[-1], data, labels)
    fs = [f0]
    n_total_gradients = 0
    
    # gradient descent loop
    for _ in tqdm(range(max_epochs)):
        w0 = ws[-1]
        grad, n_sample_gradients, J_flop_counts = J(w0, data, labels)
        
        w1 = w0 - lr * grad          # gradient descent
        w1 = proj(w1)                # projection
        ws.append(w1)
        
        n_total_gradients += n_sample_gradients

        f0 = fs[-1]
        f1, n_sample_feval, f_flop_counts = f(w1, data, labels)
        fs.append(f1)
        
        # stopping criteria, w-norm
        if np.linalg.norm(w1 - w0, ord=1) / np.linalg.norm(w0, ord=1) < w_tol:
            break
        
        # stopping criteria, f-norm
        if abs(f0 / f1 - 1) < f_tol:
            break        

    return ws, n_total_gradients


In [123]:
# optimizer definition: sgd

def sgd(data: np.ndarray, labels: np.ndarray, w0,
        f, J, proj,
        lr, w_tol, f_tol, max_epochs,
        batch_size=1, max_stall_count=50) -> Tuple[List[np.ndarray], int]:
    assert data.shape[0] == labels.shape[0]
    
    # initialization
    ws = [proj(w0)]
    f0, n_sample_feval, f_flop_counts = f(ws[-1], data, labels)
    fs = [f0]
    n_total_gradients = 0
    w_stall_counter = 0
    f_stall_counter = 0
    
    for _ in tqdm(range(max_epochs)):
        # shuffle index (in place)
        indices = np.arange(data.shape[0])
        np.random.shuffle(indices)
        
        # stochastic gradient descent loop
        for i in indices:        
            w0 = ws[-1]
            grad, n_sample_gradients, J_flop_counts = J(w0, data[i:i+1], labels[i:i+1])

            w1 = w0 - lr * grad          # gradient descent
            w1 = proj(w1)                # projection
            ws.append(w1)

            n_total_gradients += n_sample_gradients

            f0 = fs[-1]
            f1, n_sample_feval, f_flop_counts = f(w1, data, labels)
            fs.append(f1)

            # stopping criteria, w-norm
            if np.linalg.norm(w1 - w0, ord=1) / np.linalg.norm(w0, ord=1) < w_tol:
                if w_stall_counter < max_stall_count:
                    w_stall_counter += 1
                else:
                    print(f"max stall count {max_stall_count} reached for w norm")
                    break
            else:
                w_stall_counter = 0

            # stopping criteria, f-norm
            if abs(f0 / f1 - 1) < f_tol:
                if f_stall_counter < max_stall_count:
                    f_stall_counter += 1
                else:
                    print(f"max stall count {max_stall_count} reached for f norm")
                    break        
            else:
                f_stall_counter = 0
            
        else:
            # continue if the inner loop wasn't broken.
            continue
        
        # inner loop was broken, break the outer.
        break

    return ws, n_total_gradients


In [124]:
# optimizer definition: sag

def sag(data: np.ndarray, labels: np.ndarray, w0,
        f, J, proj,
        lr, w_tol, f_tol, max_epochs,
        batch_size=1, max_stall_count=50) -> Tuple[List[np.ndarray], int]:
    assert data.shape[0] == labels.shape[0]
    
    # initialization
    ws = [proj(w0)]
    f0, n_sample_feval, f_flop_counts = f(ws[-1], data, labels)
    fs = [f0]
    n_total_gradients = 0
    
    grads = np.zeros_like(data, dtype=np.float32)     # TODO: fix initialization
    avg_grad = np.zeros_like(w0, dtype=np.float32)
    
    w_stall_counter = 0
    f_stall_counter = 0
    
    for _ in tqdm(range(max_epochs)):
        # shuffle index (in place)
        indices = np.arange(data.shape[0])
        np.random.shuffle(indices)
        
        for i in indices:        
            w0 = ws[-1]
            prev_grad = grads[i]
            new_grad, n_sample_gradients, J_flop_counts = J(w0, data[i:i+1], labels[i:i+1])

            avg_grad = avg_grad + (new_grad - prev_grad) # / data.shape[0]
            w1 = w0 - lr * avg_grad      # gradient descent
            w1 = proj(w1)                # projection
            ws.append(w1)
            
            grads[i] = new_grad
            n_total_gradients += n_sample_gradients

            f0 = fs[-1]
            f1, n_sample_feval, f_flop_counts = f(w1, data, labels)
            fs.append(f1)

            # stopping criteria, w-norm
            if np.linalg.norm(w1 - w0, ord=1) / np.linalg.norm(w0, ord=1) < w_tol:
                if w_stall_counter < max_stall_count:
                    w_stall_counter += 1
                else:
                    print(f"max stall count {max_stall_count} reached for w norm")
                    break
            else:
                w_stall_counter = 0

            # stopping criteria, f-norm
            if abs(f0 / f1 - 1) < f_tol:
                if f_stall_counter < max_stall_count:
                    f_stall_counter += 1
                else:
                    print(f"max stall count {max_stall_count} reached for f norm")
                    break        
            else:
                f_stall_counter = 0
            
        else:
            # Continue if the inner loop wasn't broken.
            continue
        
        # Inner loop was broken, break the outer.
        break

    return ws, n_total_gradients

In [125]:
# problem definition

f, J = regression_builder()
proj = projection_l1_builder(l=1.0)

n = 10000
d = 20

w = np.random.random(d)
w = w / np.linalg.norm(w, ord=1)

data = np.random.random((n, d))
labels = np.dot(data, w)

w0 = np.random.random(d)
w_tol = 1e-6
f_tol = 1e-6
max_epochs = 1000


In [126]:
# main: pgd

lr = 1e-3

ws, n_total_gradients = pgd(
    data=data,
    labels=labels,
    w0=w0,
    f=f,
    J=J,
    proj=proj,
    lr=lr,
    w_tol=w_tol,
    f_tol=f_tol,
    max_epochs=max_epochs
)

print(f"w:\n {w}\n")
print(f"total sample gradient evaluations: {n_total_gradients}\n")
print(f"last 5 ws:")
[print(ws[i]) for i in range(len(ws)-5, len(ws))]


/home/nos/miniconda3/envs/projects/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  1%|▏         | 13/1000 [00:00<00:01, 939.76it/s]

w:
 [0.05767626 0.06759437 0.07666225 0.08149776 0.0239234  0.06432924
 0.00539999 0.05808347 0.00731672 0.09864687 0.03519868 0.04257602
 0.02149977 0.0174237  0.08543414 0.07448893 0.01353679 0.0706693
 0.01343755 0.08460479]

total sample gradient evaluations: 140000

last 5 ws:
[0.05767597 0.06759306 0.07666188 0.08149737 0.02392447 0.06432851
 0.00540002 0.05808348 0.0073164  0.09864657 0.0351989  0.04257633
 0.02150022 0.01742344 0.08543416 0.07448923 0.01353757 0.07067007
 0.01343691 0.08460543]
[0.05767603 0.06759396 0.07666202 0.08149755 0.02392337 0.06432894
 0.00539984 0.05808324 0.00731651 0.09864666 0.03519856 0.04257589
 0.02149967 0.01742352 0.08543396 0.07448877 0.01353675 0.07066924
 0.01343726 0.08460474]
[0.05767623 0.06759421 0.0766622  0.08149771 0.02392353 0.06432915
 0.00539999 0.05808347 0.00731668 0.09864683 0.03519871 0.04257606
 0.02149983 0.01742366 0.08543415 0.07448897 0.01353689 0.0706694
 0.01343747 0.08460487]
[0.05767624 0.06759432 0.07666222 0.0814977

[None, None, None, None, None]

In [127]:
# main: sgd

lr = 1e-2

ws, n_total_gradients = sgd(
    data=data,
    labels=labels,
    w0=w0,
    f=f,
    J=J,
    proj=proj,
    lr=lr,
    w_tol=w_tol,
    f_tol=f_tol,
    max_epochs=max_epochs,
    batch_size=1,
    max_stall_count=50
)

print(f"w:\n {w}\n")
print(f"total sample gradient evaluations: {n_total_gradients}\n")
print(f"last 5 ws:")
[print(ws[i]) for i in range(len(ws)-5, len(ws))]

  0%|          | 1/1000 [00:01<20:34,  1.24s/it]

max stall count 50 reached for w norm
w:
 [0.05767626 0.06759437 0.07666225 0.08149776 0.0239234  0.06432924
 0.00539999 0.05808347 0.00731672 0.09864687 0.03519868 0.04257602
 0.02149977 0.0174237  0.08543414 0.07448893 0.01353679 0.0706693
 0.01343755 0.08460479]

total sample gradient evaluations: 12113

last 5 ws:
[0.05767402 0.06759142 0.07666545 0.08149237 0.02392226 0.06432745
 0.00540788 0.05808295 0.00731528 0.09864568 0.03520349 0.04257458
 0.02150242 0.0174254  0.0854301  0.07448419 0.01354606 0.07067179
 0.01343477 0.08460247]
[0.05767402 0.06759142 0.07666544 0.08149237 0.02392226 0.06432744
 0.00540788 0.05808295 0.00731528 0.09864568 0.03520349 0.04257458
 0.02150242 0.0174254  0.0854301  0.07448419 0.01354606 0.07067179
 0.01343477 0.08460247]
[0.05767402 0.06759142 0.07666544 0.08149237 0.02392226 0.06432744
 0.00540788 0.05808295 0.00731528 0.09864568 0.03520349 0.04257458
 0.02150242 0.0174254  0.0854301  0.07448419 0.01354606 0.07067179
 0.01343477 0.08460247]
[0.05

[None, None, None, None, None]

In [128]:
# main: sag -- in progress; will fix if needed

"""
lr = 1e-1

ws, n_total_gradients = sag(
    data=data,
    labels=labels,
    w0=w0,
    f=f,
    J=J,
    proj=proj,
    lr=lr,
    w_tol=w_tol,
    f_tol=f_tol,
    max_epochs=max_epochs,
    batch_size=1,
    max_stall_count=50
)

print(f"w:\n {w}\n")
print(f"total sample gradient evaluations: {n_total_gradients}\n")
print(f"last 5 ws:")
[print(ws[i]) for i in range(len(ws)-5, len(ws))]
"""


'\nlr = 1e-1\n\nws, n_total_gradients = sag(\n    data=data,\n    labels=labels,\n    w0=w0,\n    f=f,\n    J=J,\n    proj=proj,\n    lr=lr,\n    w_tol=w_tol,\n    f_tol=f_tol,\n    max_epochs=max_epochs,\n    batch_size=1,\n    max_stall_count=50\n)\n\nprint(f"w:\n {w}\n")\nprint(f"total sample gradient evaluations: {n_total_gradients}\n")\nprint(f"last 5 ws:")\n[print(ws[i]) for i in range(len(ws)-5, len(ws))]\n'